# Audio Cleaner and splitter

In [1]:
import os
# os.environ['LIBROSA_CACHE_DIR'] = '/tmp/librosa_cache'
# os.environ['LIBROSA_CACHE_LEVEL'] = '30'

from glob import glob
from IPython.display import Audio  # To play sound in the notebook
from scipy.io import wavfile
from scipy.signal import butter, filtfilt
import librosa
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import asyncio
import concurrent.futures
import multiprocessing
import functools
import warnings

# disable librosa warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


In [2]:
# Load the audio file
# filename = '/home/birdo/MachineLearning/Vogelgeluiden/Turdus Merula Linnaeus/geluiden/Turdus_merula_Tre_N0110_06_short.mp3'
# y, sr = librosa.load(filename, sr=None)

# # Make some modifications to the audio data
# # For example, let's increase the volume by 3 dB
# y_modified = librosa.effects.percussive(y, margin=3.0)

# # Save the modified audio data to a new file
# output_filename = '/home/birdo/MachineLearning/clean-audio/'+'output.wav'
# wavfile.write(output_filename, sr, y_modified[:sr*2])


In [3]:
def visualize_audio_waveform(audio, sample_rate, display=True):
    plt.figure(figsize=(15, 5))
    librosa.display.waveshow(audio, sr=sample_rate)
    if display:
        plt.show()


def find_closest_start(target, chunks):
    closest = chunks[0][0]
    for chunk in chunks:
        if abs(chunk[0]-target) < abs(closest-target):
            closest = chunk[0]
    return closest


def find_closest_end(target, chunks):
    closest = chunks[len(chunks)-1][1]
    for chunk in chunks:
        if abs(chunk[1]-target) < abs(closest-target):
            closest = chunk[1]
    return closest


def filter_audio(_audio, _sr):
    low_cut = 1000  # Only allow frequencies above 1000 Hz
    high_cut = 7000  # Only allow frequencies below 7000 Hz
    b, a = butter(5, [low_cut / (_sr / 2.0),
                  high_cut / (_sr / 2.0)], btype='band')
    filtered = filtfilt(b, a, _audio)

    return filtered  # * filtered


def get_audio_parts(_audio: np.ndarray, _sr, _audio_length, length=10):
    part_count = int(_audio_length/length)
    # print(part_count)
    audio_parts = []
    part_start = 0
    part_end = 0

    for i in range(part_count):
        part_start = i*length
        part_end = part_start + length
        audio_parts.append(_audio[int(part_start*_sr):int(part_end*_sr)])

    # add last part
    part_start = part_end
    part_end = _audio_length
    if part_end - part_start > 2:
        audio_parts.append(_audio[int(part_start*_sr):int(part_end*_sr)])

    return audio_parts


def write_audio_parts(audio_parts, path, _sr):
    for i in range(len(audio_parts)):
        filepath = f"{path}_part_{i}.wav"
        wavfile.write(filepath, _sr, audio_parts[i])
    # print("done writing audio parts")

async def split_and_write_audio(bird_sound,clean_dir,bird_name,executor=None):
    # Remove extension from audio file name
    new_bird_sound = ''.join(bird_sound.split('.')[:-1])
    # Create directory if it doesn't exist
    bird_dir = os.path.join(clean_dir, bird_name)
    os.makedirs(bird_dir, exist_ok=True)
    # Create new audio file path
    new_audio_path = os.path.join(bird_dir, new_bird_sound.split('/')[-1])

    filepath = f"{new_audio_path}_part_{0}.wav"
    if os.path.exists(filepath):
        return f"Skipping {filepath}"
    
    # Load audio
    loop = asyncio.get_event_loop()
    audio, sr = await loop.run_in_executor(executor,functools.partial(librosa.load, bird_sound, sr=16000, mono=True))
    audio_length = librosa.get_duration(y=audio, sr=sr)
    audio_parts = None

    # Split audio into parts if it's longer than 10 seconds
    if audio_length <= 10:
        audio_parts = [audio]
    else:
        audio_parts = get_audio_parts(audio, sr, audio_length)

    # Write audio parts to new audio file
    await loop.run_in_executor(executor,functools.partial(write_audio_parts,audio_parts, new_audio_path, sr))
    return ""


In [4]:
# load audio file
clean_dir = "/home/birdo/MachineLearning/clean-audio/"

paths = glob('/home/birdo/MachineLearning/Vogelgeluiden/*')
bird_sounds = dict()

for path in paths:
    bird_name = path.split('/')[-1]
    bird_sounds[bird_name] = glob(path+'/geluiden/*')



# print all entries in bird_sounds dict
with concurrent.futures.ThreadPoolExecutor(max_workers=8,thread_name_prefix="birds") as pool:
    for bird_name in bird_sounds:
        split_coroutines = []
        for bird_sound in bird_sounds[bird_name]:
            split_coroutine = split_and_write_audio(bird_sound,clean_dir,bird_name,executor=pool)
            split_coroutines.append(split_coroutine)

        coroutine_count = len(split_coroutines)
        complete_count = 0
        

        for f in tqdm(asyncio.as_completed(split_coroutines), total=coroutine_count, desc=f"Splitting {bird_name} audio files",mininterval=0.01,maxinterval=10, miniters=1):
            result = await f
            complete_count += 1
            # tqdm.write(f"{complete_count}/{coroutine_count} {result}")

Splitting Turdus Merula Linnaeus audio files:   0%|          | 0/2509 [00:00<?, ?it/s]

Splitting Anas Platyrhynchos Linnaeus audio files:   0%|          | 0/1380 [00:00<?, ?it/s]

Splitting Phylloscopus Trochilus audio files:   0%|          | 0/1911 [00:00<?, ?it/s]

Splitting no bird audio files:   0%|          | 0/6695 [00:00<?, ?it/s]

Splitting Sturnus Vulgaris Linnaeus audio files:   0%|          | 0/2057 [00:00<?, ?it/s]

...

Splitting Parus Major Linnaeus audio files:   0%|          | 0/1880 [00:00<?, ?it/s]

Splitting Columba Palumbus Linnaeus audio files:   0%|          | 0/1072 [00:00<?, ?it/s]

Splitting Troglodytes Troglodytes audio files:   0%|          | 0/2218 [00:00<?, ?it/s]

Splitting Passer Domesticus audio files:   0%|          | 0/3901 [00:00<?, ?it/s]

Splitting Fringilla Coelebs Linnaeus audio files:   0%|          | 0/4321 [00:00<?, ?it/s]